In [1]:
from tqdm import tqdm
import pickle
import torch

triples = [x.split() for x in open('train2id.txt').read().splitlines()[1:]]
node2edge = {(h,t):r for h,t,r in triples}
nodes = {x.split()[0]:x.split()[1] for x in open('entity2id.txt').read().splitlines()[1:]}
edges = {x.split()[0]:x.split()[1] for x in open('relation2id.txt').read().splitlines()[1:]}

# Initialize subgraph
adm_node = list()
for node in list(nodes.items()):
    if 'hadm' in node[0]:
        adm_node.append(node[1])
subgraph_norel = [{node:list() for node in adm_node}]

#subgraph_rel = dict(adm_node)
#node_dict = list(subgraph_norel.keys())

# Build subgraph
print('start preprocessing')
level = 0
while len(triples)>0:
    queue = list()
    print('level:{}'.format(level))
    for triple in tqdm(triples):
        if triple[0] in subgraph_norel[level]:
            subgraph_norel[level][triple[0]].append(triple[1])
            flag = False
        else:
            flag = True
        if flag:
            queue.append(triple)
    print('{}/{}'.format(len(queue),len(triples)))
    new_head = list()
    for heads in list(subgraph_norel[level].values()):
        new_head+=heads
    subgraph_norel.append({k:list() for k in new_head})
    triples = queue
    level += 1

  2%|█▏                                                                 | 331362/18755503 [00:00<00:11, 1650791.24it/s]

start preprocessing
level:0


100%|█████████████████████████████████████████████████████████████████| 18755503/18755503 [00:10<00:00, 1848322.39it/s]


15929096/18755503


  0%|▎                                                                    | 78773/15929096 [00:00<00:20, 779935.67it/s]

level:1


100%|██████████████████████████████████████████████████████████████████| 15929096/15929096 [00:18<00:00, 841698.79it/s]


0/15929096


In [ ]:
def get_childs(subgraph, depth, heads):
    temp_seq = list()
    for head in heads:
        temp_seq += subgraph[depth][head]
    return temp_seq

subs = list()
max_len = 0
for head in tqdm(list(subgraph_norel[0].keys())):
    depth=0
    seq = [head]
    heads = [head]
    while depth<2:
        heads = get_childs(subgraph_norel,depth,heads)
        seq += heads
        depth+=1
    subs.append([int(x) for x in seq])
    if len(seq)>max_len:
        max_len = len(seq)

tensorized_subgraphs = torch.LongTensor([x+[-100]*(max_len-len(x)) for x in subs])
print(max_len)
print('Saving...')
torch.save(tensorized_subgraphs,'subgraph_norel')
print('Done')


In [2]:
def get_childs_withrel(subgraph, depth, heads,node2edge):
    temp_seq = list()
    temp_heads = list()
    for head in heads:
        node_set = [(head,tail) for tail in subgraph[depth][head]]
        for node_pair in node_set:
            temp_seq += ['r'+node2edge[node_pair],node_pair[1]]
        temp_heads += subgraph[depth][head]
    return temp_seq, temp_heads

subs = list()
max_len = 0
for head in tqdm(list(subgraph_norel[0].keys())):
    depth=0
    seq = [head]
    heads = [head]
    while depth<2:
        seqs, heads = get_childs_withrel(subgraph_norel,depth,heads,node2edge)
        seq += seqs
        depth+=1
    subs.append(list(map(lambda x: int(x) if 'r' not in x else -(int(x.split('r')[-1])+1),seq)))
    if len(seq)>max_len:
        max_len = len(seq)

tensorized_subgraphs = torch.LongTensor([x+[-100]*(max_len-len(x)) for x in subs])
print(max_len)
print('Saving...')
torch.save(tensorized_subgraphs,'subgraph_withrel')
print('Done')

100%|███████████████████████████████████████████████████████████████████████████| 31969/31969 [00:33<00:00, 946.77it/s]


16405
Saving...
Done


In [3]:
print(subgraph_norel[0]['71'][:5])
print(tensorized_subgraphs[0,:500])
#tensorized_subgraphs[3,:]

['72', '93519', '206215', '242177', '334606']
tensor([      7,      -4,       8,      -4,  283300,      -4,  575838,      -4,
         719045,      -4,  645680,      -4,  989663,      -4,  426595,      -4,
        1076656,      -4,  463680,      -4, 1233681,      -4,  908837,      -4,
        1283846,      -4, 1290906,      -4, 1628720,      -4, 1629404,      -4,
        1772376,      -4,  352772,      -4, 1809227,      -4,  533009,      -4,
         498708,      -4, 1858488,      -4, 1902562,      -4, 1915416,      -4,
        1955206,      -4, 2042356,      -4,  881922,      -4, 2094568,      -4,
        2098564,      -4, 2099384,      -4,   80624,      -4, 2157778,      -4,
        1888242,      -4, 1802277,      -4, 2302185,      -4, 2322619,      -4,
        1523716,      -4, 1261659,      -4,  448803,      -4,  696941,      -4,
        1189685,      -4,  927060,      -4, 1791294,      -4, 1649091,      -4,
         260145,      -4,  245836,      -4, 2620781,      -4, 2118448,    